In [ ]:
import cv2
from keras.models import load_model
import numpy as np
import os

model = load_model('emojinator.h5')

def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours,hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class],400, 250,90,90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()

0
1
2
2 0.7293688
2 0.96589434
2 0.80146307
2 0.80146307
2 0.7667115
2 0.7667115
2 0.966502
2 0.966502
2 0.9836626
2 0.9836626
2 0.9941051
2 0.9941051
2 0.99954695
2 0.99954695
2 0.99871457
2 0.99871457
2 0.99589384
2 0.99589384
2 0.9638173
2 0.9638173
2 0.90316975
2 0.90316975
2 0.9656803
2 0.9656803
2 0.8493873
2 0.8493873
2 0.96695685
2 0.96695685
2 0.71781075
2 0.71781075
2 0.9662153
2 0.9662153
2 0.89275694
2 0.89275694
2 0.968733
2 0.968733
2 0.9335789
2 0.9335789
2 0.75612265
2 0.75612265
2 0.96710044
2 0.96710044
2 0.9569544
2 0.9569544
2 0.9666528
2 0.9666528
2 0.9664334
2 0.9664334
2 0.90156937
2 0.90156937
2 0.95418626
2 0.95418626
2 0.9741945
2 0.9741945
2 0.98972535
2 0.98972535
2 0.98921424
2 0.98921424
2 0.9792159
2 0.9792159
2 0.89076823
2 0.89076823
2 0.96615726
2 0.96615726
2 0.9679836
2 0.9679836
2 0.7366903
2 0.7366903
2 0.9665232
2 0.9665232
2 0.9981641
2 0.9981641
2 0.9054273
2 0.9054273
2 0.9664734
2 0.9664734
2 0.98061943
2 0.98061943
2 0.968215
2 0.968215
2 0.9

1 0.74243605
1 0.74243605
1 0.9918733
1 0.9302053
1 0.9302053
1 0.9927245
1 0.9927245
1 0.98764986
1 0.98764986
1 0.81634635
1 0.81634635
1 0.8380874
1 0.9969098
1 0.9969098
1 0.9961022
1 0.9961022
1 0.9896035
1 0.9896035
1 0.9718471
1 0.9718471
1 0.8659643
1 0.8659643
1 0.9577099
1 0.9577099
2 0.48291332
2 0.48291332
2 0.66428673
1 0.4680124
1 0.4680124
2 0.45961672
2 0.45961672
2 0.6627453
2 0.6627453
1 0.52328384
1 0.52328384
2 0.50789595
2 0.50789595
1 0.50076056
1 0.50076056
2 0.68246317
1 0.81915706
1 0.81915706
1 0.5042204
1 0.5042204
1 0.9611821
1 0.9611821
1 0.967388
1 0.967388
1 0.97293925
1 0.97293925
1 0.9499396
1 0.9499396
1 0.9611155
1 0.9611155
1 0.9829894
1 0.9829894
1 0.9537296
1 0.9537296
1 0.95543814
1 0.95543814
1 0.9610855
1 0.9610855
1 0.97393924
1 0.97393924
1 0.9771238
1 0.9771238
1 0.97263724
1 0.97263724
1 0.9468644
1 0.9468644
2 0.9827633
2 0.9827633
2 0.98081654
2 0.98081654
2 0.9619361
2 0.9619361
2 0.7053782
2 0.7053782
1 0.62829244
1 0.62829244
1 0.623913